# Quantum-Inspired Recommender System

This notebook demonstrates how concepts from quantum mechanics can be applied to build recommender systems. We'll explore:

1. **Complex Hilbert Space Embeddings** - Representing users and items as quantum states
2. **Quantum Interference** - How multiple paths affect recommendations
3. **Density Matrices** - Capturing uncertainty in user preferences
4. **Born Rule** - Converting quantum amplitudes to probabilities

## Background

In quantum mechanics, particles exist in superposition states and can interfere with themselves. These mathematical concepts can provide interesting alternatives to classical collaborative filtering approaches.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install numpy scipy matplotlib --break-system-packages

In [ ]:
import numpy as np
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

## 1. The Quantum-Inspired Recommender Class

Our recommender system uses complex-valued embeddings in a Hilbert space. Each user and item is represented as a complex vector, similar to quantum state vectors.

In [ ]:
class QuantumInspiredRecommender:
    """
    A recommender system using quantum-inspired methods:
    1. Complex Hilbert space embeddings
    2. Quantum interference effects
    3. Density matrix representation of user preferences
    """
    
    def __init__(self, n_users: int, n_items: int, embedding_dim: int = 10):
        """
        Initialize the quantum-inspired recommender
        
        Args:
            n_users: Number of users
            n_items: Number of items
            embedding_dim: Dimension of the Hilbert space (complex vector space)
        """
        self.n_users = n_users
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        
        # Complex embeddings in Hilbert space (quantum states)
        # Users and items are represented as complex vectors
        self.user_embeddings = self._initialize_complex_embeddings(n_users, embedding_dim)
        self.item_embeddings = self._initialize_complex_embeddings(n_items, embedding_dim)
        
        # Density matrices for users (mixed quantum states)
        self.user_density_matrices = None
        
    def _initialize_complex_embeddings(self, n: int, dim: int) -> np.ndarray:
        """
        Initialize complex-valued embeddings (quantum state vectors)
        Each embedding is normalized to unit length (like quantum state vectors)
        """
        # Random complex numbers
        real_part = np.random.randn(n, dim) * 0.1
        imag_part = np.random.randn(n, dim) * 0.1
        embeddings = real_part + 1j * imag_part
        
        # Normalize to unit vectors (quantum normalization)
        norms = np.sqrt(np.sum(np.abs(embeddings)**2, axis=1, keepdims=True))
        return embeddings / norms
    
    def quantum_inner_product(self, vec1: np.ndarray, vec2: np.ndarray) -> complex:
        """
        Compute quantum inner product (bra-ket notation: <vec1|vec2>)
        This is the complex conjugate of vec1 dotted with vec2
        """
        return np.dot(np.conj(vec1), vec2)
    
    def compute_probability_amplitude(self, user_idx: int, item_idx: int) -> complex:
        """
        Compute probability amplitude for user-item interaction
        In quantum mechanics, probability = |amplitude|^2
        """
        user_state = self.user_embeddings[user_idx]
        item_state = self.item_embeddings[item_idx]
        
        # Quantum inner product gives the probability amplitude
        amplitude = self.quantum_inner_product(user_state, item_state)
        return amplitude
    
    def predict_with_interference(self, user_idx: int, item_idx: int, 
                                   context_items: List[int] = None) -> float:
        """
        Predict rating using quantum interference effects
        
        Multiple paths (through context items) can interfere constructively 
        or destructively, similar to the double-slit experiment
        """
        # Direct amplitude
        direct_amplitude = self.compute_probability_amplitude(user_idx, item_idx)
        
        if context_items is None or len(context_items) == 0:
            # No interference, just direct path
            probability = np.abs(direct_amplitude)**2
            return float(probability * 5)  # Scale to 0-5 rating
        
        # Compute interference from multiple paths
        total_amplitude = direct_amplitude
        
        for context_item in context_items:
            # Path through context item (two-step process)
            amp1 = self.compute_probability_amplitude(user_idx, context_item)
            amp2 = self.quantum_inner_product(
                self.item_embeddings[context_item], 
                self.item_embeddings[item_idx]
            )
            # Multiply amplitudes for sequential events
            total_amplitude += amp1 * amp2 * 0.3  # Weight factor
        
        # Born rule: probability = |amplitude|^2
        probability = np.abs(total_amplitude)**2
        
        # Scale to rating (0-5)
        return float(np.clip(probability * 5, 0, 5))
    
    def create_density_matrix(self, user_idx: int, rated_items: Dict[int, float]) -> np.ndarray:
        """
        Create a density matrix representation of user preferences
        
        Density matrices represent mixed quantum states and can capture
        uncertainty in user preferences
        """
        # Start with zero matrix
        density_matrix = np.zeros((self.embedding_dim, self.embedding_dim), dtype=complex)
        
        # Add contribution from each rated item
        total_weight = 0
        for item_idx, rating in rated_items.items():
            item_state = self.item_embeddings[item_idx]
            # Weight by rating (normalized)
            weight = rating / 5.0
            # Outer product: |ψ⟩⟨ψ| (projection operator)
            density_matrix += weight * np.outer(item_state, np.conj(item_state))
            total_weight += weight
        
        # Normalize
        if total_weight > 0:
            density_matrix /= total_weight
        
        return density_matrix
    
    def predict_with_density_matrix(self, user_idx: int, item_idx: int,
                                     rated_items: Dict[int, float]) -> float:
        """
        Predict rating using density matrix formalism
        
        This captures mixed states and uncertainty in user preferences
        """
        # Create density matrix from user's rating history
        rho = self.create_density_matrix(user_idx, rated_items)
        
        # Item state
        item_state = self.item_embeddings[item_idx]
        
        # Expectation value: Tr(ρ|ψ⟩⟨ψ|)
        projection = np.outer(item_state, np.conj(item_state))
        expectation = np.trace(rho @ projection)
        
        # Scale to rating
        return float(np.clip(np.real(expectation) * 5, 0, 5))
    
    def train(self, ratings: List[Tuple[int, int, float]], 
              epochs: int = 100, learning_rate: float = 0.01, verbose: bool = True):
        """
        Train the quantum-inspired embeddings using gradient descent
        
        Args:
            ratings: List of (user_idx, item_idx, rating) tuples
            epochs: Number of training epochs
            learning_rate: Learning rate for optimization
            verbose: Whether to print training progress
        """
        if verbose:
            print("Training quantum-inspired recommender...")
        
        losses = []
        
        for epoch in range(epochs):
            total_loss = 0
            
            for user_idx, item_idx, true_rating in ratings:
                # Forward pass
                amplitude = self.compute_probability_amplitude(user_idx, item_idx)
                predicted_rating = np.abs(amplitude)**2 * 5
                
                # Compute loss (MSE)
                error = predicted_rating - true_rating
                total_loss += error**2
                
                # Gradient descent update (simplified)
                user_state = self.user_embeddings[user_idx]
                item_state = self.item_embeddings[item_idx]
                
                # Gradient approximation
                gradient_scale = 2 * error * np.abs(amplitude) * 5
                
                # Update embeddings (maintaining normalization)
                self.user_embeddings[user_idx] -= learning_rate * gradient_scale * np.conj(item_state)
                self.item_embeddings[item_idx] -= learning_rate * gradient_scale * np.conj(user_state)
                
                # Re-normalize (quantum states must be unit vectors)
                self.user_embeddings[user_idx] /= np.linalg.norm(self.user_embeddings[user_idx])
                self.item_embeddings[item_idx] /= np.linalg.norm(self.item_embeddings[item_idx])
            
            avg_loss = total_loss / len(ratings)
            losses.append(avg_loss)
            
            if verbose and epoch % 20 == 0:
                print(f"Epoch {epoch}, Average Loss: {avg_loss:.4f}")
        
        return losses

print("QuantumInspiredRecommender class defined successfully!")

## 2. Create and Initialize the Recommender

Let's create a recommender system with 20 users and 30 items, using 8-dimensional complex embeddings.

In [ ]:
# System parameters
n_users = 20
n_items = 30
embedding_dim = 8

# Create the recommender
recommender = QuantumInspiredRecommender(n_users, n_items, embedding_dim)

print(f"Created quantum-inspired recommender:")
print(f"  Users: {n_users}")
print(f"  Items: {n_items}")
print(f"  Embedding dimension: {embedding_dim}")
print(f"\nUser embeddings shape: {recommender.user_embeddings.shape}")
print(f"Item embeddings shape: {recommender.item_embeddings.shape}")
print(f"\nEmbeddings are complex-valued: {np.iscomplexobj(recommender.user_embeddings)}")

### Verify Quantum Properties

Quantum state vectors must be normalized (have unit length). Let's verify this property:

In [ ]:
# Check normalization (quantum states should have norm = 1)
user_norms = np.linalg.norm(recommender.user_embeddings, axis=1)
item_norms = np.linalg.norm(recommender.item_embeddings, axis=1)

print("Quantum State Normalization Check:")
print(f"User state norms - Min: {user_norms.min():.6f}, Max: {user_norms.max():.6f}")
print(f"Item state norms - Min: {item_norms.min():.6f}, Max: {item_norms.max():.6f}")
print(f"\nAll states properly normalized: {np.allclose(user_norms, 1.0) and np.allclose(item_norms, 1.0)}")

# Show example of a complex embedding
print(f"\nExample user embedding (User 0):")
print(recommender.user_embeddings[0])
print(f"\nMagnitude: {np.abs(recommender.user_embeddings[0])}")
print(f"Phase: {np.angle(recommender.user_embeddings[0])}")

## 3. Generate Synthetic Training Data

We'll create synthetic user-item ratings for training.

In [ ]:
# Generate synthetic ratings
np.random.seed(42)
n_ratings = 200

ratings = []
for _ in range(n_ratings):
    user = np.random.randint(0, n_users)
    item = np.random.randint(0, n_items)
    rating = np.random.uniform(1, 5)
    ratings.append((user, item, rating))

print(f"Generated {len(ratings)} synthetic ratings")
print(f"\nFirst 10 ratings:")
for i, (u, it, r) in enumerate(ratings[:10]):
    print(f"  {i+1}. User {u} rated Item {it}: {r:.2f} stars")

## 4. Train the Model

Now we'll train the quantum-inspired embeddings using gradient descent.

In [ ]:
# Train the model
losses = recommender.train(ratings, epochs=100, learning_rate=0.05, verbose=True)

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses, linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Average Loss (MSE)', fontsize=12)
plt.title('Training Loss Over Time', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nFinal loss: {losses[-1]:.4f}")

## 5. Demonstration 1: Quantum Interference Effects

In quantum mechanics, particles can take multiple paths, and these paths interfere with each other. We apply this concept to recommendations: considering different "context items" (alternative paths) affects the final prediction through interference.

In [ ]:
# Select a test user and item
test_user = 0
test_item = 5

print(f"Predicting rating for User {test_user} and Item {test_item}")
print("="*60)

# Prediction with no interference (direct path only)
pred_direct = recommender.predict_with_interference(test_user, test_item, [])
print(f"\n1. Direct path only (no interference):")
print(f"   Predicted rating: {pred_direct:.3f} stars")

# Prediction with different context items (interference)
context_sets = [
    [1, 2],
    [10, 15],
    [1, 2, 10],
    [5, 12, 18, 25]
]

print(f"\n2. With interference (through context items):")
for i, context in enumerate(context_sets, 1):
    pred = recommender.predict_with_interference(test_user, test_item, context)
    print(f"   Context {context}: {pred:.3f} stars")

print("\n" + "="*60)
print("Notice how different context items produce different predictions!")
print("This is quantum interference in action.")

### Visualize Interference Effects

Let's visualize how predictions change as we vary the context items:

In [ ]:
# Test multiple items with different context scenarios
test_items = range(0, 15)
predictions_no_context = []
predictions_context1 = []
predictions_context2 = []

for item in test_items:
    pred_no = recommender.predict_with_interference(test_user, item, [])
    pred_c1 = recommender.predict_with_interference(test_user, item, [1, 2])
    pred_c2 = recommender.predict_with_interference(test_user, item, [10, 15])
    
    predictions_no_context.append(pred_no)
    predictions_context1.append(pred_c1)
    predictions_context2.append(pred_c2)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(test_items, predictions_no_context, 'o-', label='No interference', linewidth=2, markersize=8)
plt.plot(test_items, predictions_context1, 's-', label='Context items [1, 2]', linewidth=2, markersize=8)
plt.plot(test_items, predictions_context2, '^-', label='Context items [10, 15]', linewidth=2, markersize=8)

plt.xlabel('Item ID', fontsize=12)
plt.ylabel('Predicted Rating', fontsize=12)
plt.title(f'Quantum Interference Effects for User {test_user}', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.ylim(0, 5.5)
plt.tight_layout()
plt.show()

print("The different lines show how context items create interference patterns!")

## 6. Demonstration 2: Density Matrix Approach

Density matrices represent "mixed quantum states" and are useful for capturing uncertainty in user preferences based on their rating history.

In [ ]:
# Create a user's rating history
user_history = {
    2: 5.0,   # User loved item 2
    7: 4.5,   # User liked item 7
    12: 2.0,  # User disliked item 12
    15: 4.0,  # User liked item 15
}

print(f"User {test_user}'s rating history:")
print("="*40)
for item, rating in user_history.items():
    stars = '★' * int(rating) + '☆' * (5 - int(rating))
    print(f"  Item {item:2d}: {rating:.1f} {stars}")

# Create density matrix
rho = recommender.create_density_matrix(test_user, user_history)

print(f"\nDensity matrix shape: {rho.shape}")
print(f"Is Hermitian: {np.allclose(rho, rho.conj().T)}")
print(f"Trace (should be ≈1): {np.trace(rho):.6f}")

# Visualize the density matrix
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Real part
im1 = ax1.imshow(np.real(rho), cmap='RdBu', aspect='auto')
ax1.set_title('Density Matrix - Real Part', fontsize=12, fontweight='bold')
ax1.set_xlabel('Dimension')
ax1.set_ylabel('Dimension')
plt.colorbar(im1, ax=ax1)

# Imaginary part
im2 = ax2.imshow(np.imag(rho), cmap='RdBu', aspect='auto')
ax2.set_title('Density Matrix - Imaginary Part', fontsize=12, fontweight='bold')
ax2.set_xlabel('Dimension')
ax2.set_ylabel('Dimension')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

In [ ]:
# Predict for unrated items using density matrix
print("\nPredictions for unrated items (using density matrix):")
print("="*60)

test_items_dm = [3, 8, 10, 16, 20, 25]
for item in test_items_dm:
    pred = recommender.predict_with_density_matrix(test_user, item, user_history)
    stars = '★' * int(pred) + '☆' * (5 - int(pred))
    print(f"  Item {item:2d}: {pred:.3f} {stars}")

## 7. Demonstration 3: Visualizing Quantum States

Let's visualize the complex embeddings by projecting them to 2D.

In [ ]:
# Select users and items to visualize
n_vis = 10
user_indices = range(min(n_vis, n_users))
item_indices = range(min(n_vis, n_items))

# Project to 2D (take first two dimensions)
user_real = np.real(recommender.user_embeddings[user_indices, :2])
user_imag = np.imag(recommender.user_embeddings[user_indices, :2])

item_real = np.real(recommender.item_embeddings[item_indices, :2])
item_imag = np.imag(recommender.item_embeddings[item_indices, :2])

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# User states
axes[0, 0].scatter(user_real[:, 0], user_real[:, 1], c='blue', s=100, alpha=0.6, edgecolors='black', linewidth=1.5)
axes[0, 0].scatter(user_imag[:, 0], user_imag[:, 1], c='red', s=100, alpha=0.6, edgecolors='black', linewidth=1.5, marker='s')
axes[0, 0].set_title('User Quantum States (Real=circles, Imag=squares)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Dimension 1')
axes[0, 0].set_ylabel('Dimension 2')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='k', linewidth=0.5)
axes[0, 0].axvline(x=0, color='k', linewidth=0.5)

# Item states
axes[0, 1].scatter(item_real[:, 0], item_real[:, 1], c='green', s=100, alpha=0.6, edgecolors='black', linewidth=1.5)
axes[0, 1].scatter(item_imag[:, 0], item_imag[:, 1], c='orange', s=100, alpha=0.6, edgecolors='black', linewidth=1.5, marker='s')
axes[0, 1].set_title('Item Quantum States (Real=circles, Imag=squares)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Dimension 1')
axes[0, 1].set_ylabel('Dimension 2')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0, color='k', linewidth=0.5)
axes[0, 1].axvline(x=0, color='k', linewidth=0.5)

# Complex plane visualization for users (magnitude and phase)
user_mag = np.abs(recommender.user_embeddings[user_indices, 0])
user_phase = np.angle(recommender.user_embeddings[user_indices, 0])

# Convert to cartesian for visualization
user_x = user_mag * np.cos(user_phase)
user_y = user_mag * np.sin(user_phase)

axes[1, 0].scatter(user_x, user_y, c=range(len(user_indices)), cmap='viridis', s=150, alpha=0.7, edgecolors='black', linewidth=1.5)
for i, (x, y) in enumerate(zip(user_x, user_y)):
    axes[1, 0].arrow(0, 0, x*0.9, y*0.9, head_width=0.03, head_length=0.02, fc='gray', ec='gray', alpha=0.3)
    axes[1, 0].text(x, y, f'U{i}', fontsize=9, ha='center', va='center')

axes[1, 0].set_title('Users in Complex Plane (Dim 0)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Real Part')
axes[1, 0].set_ylabel('Imaginary Part')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='k', linewidth=0.5)
axes[1, 0].axvline(x=0, color='k', linewidth=0.5)
axes[1, 0].set_aspect('equal')

# Complex plane visualization for items
item_mag = np.abs(recommender.item_embeddings[item_indices, 0])
item_phase = np.angle(recommender.item_embeddings[item_indices, 0])

item_x = item_mag * np.cos(item_phase)
item_y = item_mag * np.sin(item_phase)

axes[1, 1].scatter(item_x, item_y, c=range(len(item_indices)), cmap='plasma', s=150, alpha=0.7, edgecolors='black', linewidth=1.5, marker='D')
for i, (x, y) in enumerate(zip(item_x, item_y)):
    axes[1, 1].arrow(0, 0, x*0.9, y*0.9, head_width=0.03, head_length=0.02, fc='gray', ec='gray', alpha=0.3)
    axes[1, 1].text(x, y, f'I{i}', fontsize=9, ha='center', va='center')

axes[1, 1].set_title('Items in Complex Plane (Dim 0)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Real Part')
axes[1, 1].set_ylabel('Imaginary Part')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(y=0, color='k', linewidth=0.5)
axes[1, 1].axvline(x=0, color='k', linewidth=0.5)
axes[1, 1].set_aspect('equal')

plt.tight_layout()
plt.show()

print("Each point represents a quantum state in complex space!")
print("The arrows show magnitude and phase (angle) of the complex numbers.")

## 8. Comparison of Different Methods

Let's compare predictions from different quantum-inspired approaches for the same user-item pairs.

In [ ]:
import pandas as pd

# Test multiple user-item pairs
test_pairs = [
    (0, 5),
    (1, 10),
    (2, 15),
    (3, 20),
    (5, 8),
]

results = []

for user, item in test_pairs:
    # Direct prediction
    pred_direct = recommender.predict_with_interference(user, item, [])
    
    # With interference
    pred_interference = recommender.predict_with_interference(user, item, [2, 7])
    
    # Density matrix (using a generic history)
    generic_history = {2: 5.0, 7: 4.5, 12: 2.0}
    pred_density = recommender.predict_with_density_matrix(user, item, generic_history)
    
    results.append({
        'User': user,
        'Item': item,
        'Direct': pred_direct,
        'Interference': pred_interference,
        'Density Matrix': pred_density,
    })

df = pd.DataFrame(results)
print("\nComparison of Quantum-Inspired Prediction Methods")
print("="*70)
print(df.to_string(index=False))
print("\nNotice how different methods can give quite different predictions!")

In [ ]:
# Visualize the comparison
x = np.arange(len(test_pairs))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar(x - width, df['Direct'], width, label='Direct', alpha=0.8)
bars2 = ax.bar(x, df['Interference'], width, label='Interference', alpha=0.8)
bars3 = ax.bar(x + width, df['Density Matrix'], width, label='Density Matrix', alpha=0.8)

ax.set_xlabel('User-Item Pairs', fontsize=12)
ax.set_ylabel('Predicted Rating', fontsize=12)
ax.set_title('Comparison of Quantum-Inspired Prediction Methods', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f"U{u},I{i}" for u, i in test_pairs])
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 6)

plt.tight_layout()
plt.show()

## 9. Interactive Exploration

Now you can experiment with the quantum recommender system yourself!

In [ ]:
# Interactive prediction function
def make_prediction(user_id, item_id, context_items=None, method='interference'):
    """
    Make a prediction for a user-item pair
    
    Args:
        user_id: User index (0 to n_users-1)
        item_id: Item index (0 to n_items-1)
        context_items: List of item indices for interference (optional)
        method: 'interference' or 'density'
    """
    if user_id >= n_users or user_id < 0:
        print(f"Error: user_id must be between 0 and {n_users-1}")
        return
    
    if item_id >= n_items or item_id < 0:
        print(f"Error: item_id must be between 0 and {n_items-1}")
        return
    
    print(f"\nPrediction for User {user_id} and Item {item_id}")
    print("="*50)
    
    if method == 'interference':
        if context_items is None:
            context_items = []
        pred = recommender.predict_with_interference(user_id, item_id, context_items)
        print(f"Method: Quantum Interference")
        print(f"Context items: {context_items if context_items else 'None'}")
    else:
        # Use a generic rating history for density matrix
        history = {2: 5.0, 7: 4.5, 12: 2.0}
        pred = recommender.predict_with_density_matrix(user_id, item_id, history)
        print(f"Method: Density Matrix")
    
    stars = '★' * int(pred) + '☆' * (5 - int(pred))
    print(f"\nPredicted Rating: {pred:.3f} {stars}")
    return pred

# Example usage
print("Try making predictions! Examples:")
print("  make_prediction(0, 5)")
print("  make_prediction(0, 5, context_items=[1, 2, 10])")
print("  make_prediction(3, 15, method='density')")

# Run some examples
make_prediction(0, 5)
make_prediction(0, 5, context_items=[1, 2, 10])
make_prediction(3, 15, method='density')

## 10. Summary and Key Takeaways

### What We've Demonstrated:

1. **Complex Hilbert Space Embeddings**
   - Users and items represented as complex vectors
   - Normalized to unit length (quantum state property)
   - Richer representation than real-valued embeddings

2. **Quantum Interference**
   - Multiple recommendation paths can interfere
   - Context items significantly affect predictions
   - Analogous to quantum superposition

3. **Density Matrices**
   - Represent mixed quantum states
   - Capture uncertainty in preferences
   - Based on user rating history

4. **Born Rule**
   - Probability = |amplitude|²
   - Converts quantum amplitudes to ratings

### Advantages Over Classical Methods:

- **Richer representations**: Complex numbers carry more information (magnitude + phase)
- **Natural uncertainty handling**: Quantum formalism built for probabilistic reasoning
- **Interference effects**: Capture subtle relationships between items
- **Theoretical foundation**: Grounded in quantum probability theory

### Potential Applications:

- Capturing context-dependent preferences
- Modeling uncertainty in user behavior
- Finding non-obvious item relationships
- Combining multiple recommendation signals

## 11. Further Experiments

Try these experiments to explore further:

1. **Vary the embedding dimension**: How does performance change with different Hilbert space dimensions?
2. **Test with real data**: Load a real recommendation dataset (MovieLens, Amazon, etc.)
3. **Compare to classical methods**: Implement standard matrix factorization and compare
4. **Tune interference weights**: Modify the 0.3 weight factor in the interference calculation
5. **Implement quantum entanglement**: Model user-user or item-item entanglement
6. **Add quantum measurement**: Implement projection operators for preference categories

In [ ]:
# Space for your experiments!
print("Happy experimenting with quantum-inspired recommendations!")